In [1]:
import matlab.engine
import numpy as np
import h5py
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import yaml
import signal

import LokiCallerV2 as env


In [2]:
###* Experimental Data CO 2

schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "fracCO2ini_relO2": "frac_CO2",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "Tgas":         "Tgas",
    "O mean":       "O",
    "CO mean":      "CO",
    "MeanLossProb": "gamma_exp"
}

file_path = "Experimental_data_CO_2/Data_CO2_TD.xlsx"

pd_exp = pd.read_excel(file_path)
pd_result_exp = pd.DataFrame()

for old, new in schema.items():
    if old in pd_exp.columns:
        pd_result_exp[new] = pd_exp[old]


pd_result_exp["Tw"] = pd_result_exp["Tw"] + 273.15 
pd_result_exp["current"] = pd_result_exp["current"] * 1e-3
pd_result_exp["Tnw"] = pd_result_exp["Tw"] + 0.45 * (pd_result_exp["Tgas"] - pd_result_exp["Tw"])
pd_result_exp["frac_O2"] = 1.0 - pd_result_exp["frac_CO2"]

print(pd_result_exp.head())
print(pd_result_exp.shape)



   pressure  current  frac_CO2      Tw        Tgas  gamma_exp         Tnw  \
0       0.4     0.04       0.0  323.15  360.560032   0.001819  339.984514   
1       0.6     0.04       0.0  323.15  372.733684   0.000995  345.462658   
2       0.8     0.04       0.0  323.15  384.630566   0.000841  350.816255   
3       1.0     0.04       0.0  323.15  396.252108   0.000775  356.045949   
4       1.5     0.04       0.0  323.15  424.110735   0.000827  368.582331   

   frac_O2  
0      1.0  
1      1.0  
2      1.0  
3      1.0  
4      1.0  
(332, 8)


In [3]:
base_path = 'LoKI_v3/Code'
input_file = 'CO2_novib_28mar2025_mod_caller.in'
chem_file = 'Oxygen/oxygen_novib_thermalflux_effective.chem'

eng = matlab.engine.start_matlab()
eng.cd(base_path, nargout=0)



In [4]:

class MySimulator(env.PhysicalSimulator):
    
    
    def modify_input_data(self, params):
        
        pressure, Tgas, Tnw, Tw, frac_O, frac_CO2, folder_str = params
        
        self.data['workingConditions']['gasPressure'] = float(pressure)
        self.data['workingConditions']['gasTemperature'] = float(Tgas)
        self.data['workingConditions']['nearWallTemperature'] = float(Tnw)
        self.data['workingConditions']['wallTemperature'] = float(Tw)
        
        fractions = [f'O = 0', f'CO2 = {float(frac_CO2)}', 'CO = 0', f'O2 = {float(frac_O)}']
        self.data['electronKinetics']['gasProperties']['fraction'] = fractions
        self.data['output']['folder'] = folder_str
    
    
    
    def modify_params_chem(self, str_vec, content_vec, params_mod):
        
        idx_interest = str_vec.index(params_mod[0])
        content_vec[idx_interest]['rate_expr'][0] = str(params_mod[1])
        
        return content_vec
    


sim = MySimulator(eng, base_path, input_file, chem_file)   


In [5]:
print(len(pd_result_exp['pressure']))

332


In [6]:


def _timeout_handler(signum, frame):
    raise TimeoutError()

signal.signal(signal.SIGALRM, _timeout_handler)

threshold = 20*60  # seconds

results = []

# ne_min = 1e15
# ne_max = 1e16

ne_min = 3e15
ne_max = 12.5e15


# for counter in range(160, 180):
for counter in range(32, 50):
    pressure = pd_result_exp['pressure'][counter] * 133.3223684
    Tw       = pd_result_exp["Tw"][counter]
    Tnw      = pd_result_exp['Tnw'][counter]
    Tgas     = pd_result_exp["Tgas"][counter]
    frac_CO2 = pd_result_exp['frac_CO2'][counter]
    frac_O2  = 1.0 - frac_CO2
    folder_str  = f"sim2_{counter}"
    current_val = pd_result_exp['current'][counter]
    gamma_exp   = pd_result_exp['gamma_exp'][counter]

    params_tuple = (pressure, Tgas, Tnw, Tw, frac_O2, frac_CO2, folder_str)
    params_chem  = ['O(3P) + wall -> 0.5O2(X)', gamma_exp]
    
    print(f"Output: {folder_str}")
    
    try:
        signal.alarm(threshold)
        results_den, names_den = sim.solver_one_point(params_tuple, params_chem, current_val, ne_min, ne_max)
        results.append(results_den)

    except TimeoutError:
        print()
        print(f"Iteration {counter} timed out after {threshold}s — skipping.")
        print()
    finally:
        signal.alarm(0)

print("Done.")


Output: sim2_1
Call: 1 I: 0.012394545582994689 I_exp: 0.04
Call: 2 I: 0.05321983781134485 I_exp: 0.04
Call: 3 I: 0.03260829918887613 I_exp: 0.04
Call: 4 I: 0.0395869383584775 I_exp: 0.04
Output: sim2_2
Call: 5 I: 0.01206766149074175 I_exp: 0.04
Call: 6 I: 0.05196410633893889 I_exp: 0.04
Call: 7 I: 0.03144245140258184 I_exp: 0.04
Call: 8 I: 0.04019084951092281 I_exp: 0.04
Output: sim2_3
Call: 9 I: 0.0118732097355981 I_exp: 0.04
Call: 10 I: 0.0511308379913117 I_exp: 0.04
Call: 11 I: 0.030975125290260152 I_exp: 0.04
Call: 12 I: 0.04016981387120965 I_exp: 0.04
Output: sim2_4
Call: 13 I: 0.011670206242093735 I_exp: 0.04
Call: 14 I: 0.04983125778122898 I_exp: 0.04
Call: 15 I: 0.030638047234262066 I_exp: 0.04
Call: 16 I: 0.0397322264040198 I_exp: 0.04
Output: sim2_5
Call: 17 I: 0.011540847338418609 I_exp: 0.04
Call: 18 I: 0.049045838486670004 I_exp: 0.04
Call: 19 I: 0.030068900154151243 I_exp: 0.04
Call: 20 I: 0.039926833751550345 I_exp: 0.04
Output: sim2_6
Call: 21 I: 0.01143466172910078 I_e